# 嵌入数据详细分析 - 理解唯一值数量

本笔记本专门解答：为什么 embedding 维度的唯一值数量会少于 movie_id 数量？

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

pd.set_option('display.max_columns', None)
plt.style.use('default')
sns.set_palette("husl")

In [ ]:
# 读取数据
data_dir = Path('.')
df_image = pd.read_parquet(data_dir / 'image_embeddings_50d.parquet')
df_text = pd.read_parquet(data_dir / 'text_embeddings_50d.parquet')

print(f"图像嵌入数据形状: {df_image.shape}")
print(f"文本嵌入数据形状: {df_text.shape}")
print(f"\n图像数据列: {list(df_image.columns[:5])}...")
print(f"文本数据列: {list(df_text.columns[:5])}...")

## 1. 理解"唯一值数量"的含义

**关键点**：`nunique()` 统计的是**单个列**中有多少个不同的值，而不是整个嵌入向量的唯一性。

In [ ]:
# 检查 movie_id 的唯一性
print("=" * 80)
print("Movie ID 分析")
print("=" * 80)
print(f"\n图像数据:")
print(f"  总行数: {len(df_image)}")
print(f"  唯一 movie_id 数量: {df_image['movie_id'].nunique()}")
print(f"  是否有重复: {df_image['movie_id'].duplicated().any()}")

print(f"\n文本数据:")
print(f"  总行数: {len(df_text)}")
print(f"  唯一 movie_id 数量: {df_text['movie_id'].nunique()}")
print(f"  是否有重复: {df_text['movie_id'].duplicated().any()}")

## 2. 检查整个嵌入向量的唯一性

现在让我们检查**完整的嵌入向量**（所有50维组合）是否唯一：

In [ ]:
def check_embedding_uniqueness(df, name):
    """检查嵌入向量的唯一性"""
    print("=" * 80)
    print(f"{name} - 嵌入向量唯一性分析")
    print("=" * 80)
    
    # 获取所有嵌入维度列
    emb_cols = [col for col in df.columns if col.startswith('emb_')]
    print(f"\n嵌入维度数: {len(emb_cols)}")
    
    # 提取嵌入矩阵
    embeddings = df[emb_cols].values
    print(f"嵌入矩阵形状: {embeddings.shape}")
    
    # 方法1: 使用 pandas 的 drop_duplicates 检查
    unique_embeddings = df[emb_cols].drop_duplicates()
    print(f"\n唯一嵌入向量数量: {len(unique_embeddings)}")
    print(f"总嵌入向量数量: {len(df)}")
    print(f"重复的嵌入向量数量: {len(df) - len(unique_embeddings)}")
    
    if len(df) == len(unique_embeddings):
        print("\n✅ 每个电影都有唯一的嵌入向量！")
    else:
        print(f"\n⚠️  有 {len(df) - len(unique_embeddings)} 个重复的嵌入向量")
        
        # 找出重复的嵌入
        duplicated_mask = df[emb_cols].duplicated(keep=False)
        duplicated_movies = df[duplicated_mask]['movie_id'].values
        print(f"   涉及的 movie_id: {duplicated_movies[:10]}..." if len(duplicated_movies) > 10 else f"   涉及的 movie_id: {duplicated_movies}")
    
    return embeddings, emb_cols

# 检查图像嵌入
image_embeddings, image_emb_cols = check_embedding_uniqueness(df_image, "图像嵌入")

In [ ]:
# 检查文本嵌入
text_embeddings, text_emb_cols = check_embedding_uniqueness(df_text, "文本嵌入")

## 3. 为什么单个维度的唯一值会少？

让我们深入分析：

In [ ]:
print("=" * 80)
print("单个维度唯一值分析")
print("=" * 80)

# 统计每个维度的唯一值数量
unique_counts_image = [df_image[col].nunique() for col in image_emb_cols]
unique_counts_text = [df_text[col].nunique() for col in text_emb_cols]

print(f"\n图像嵌入 - 各维度唯一值统计:")
print(f"  最小值: {min(unique_counts_image)}")
print(f"  最大值: {max(unique_counts_image)}")
print(f"  平均值: {np.mean(unique_counts_image):.2f}")
print(f"  中位数: {np.median(unique_counts_image):.2f}")

print(f"\n文本嵌入 - 各维度唯一值统计:")
print(f"  最小值: {min(unique_counts_text)}")
print(f"  最大值: {max(unique_counts_text)}")
print(f"  平均值: {np.mean(unique_counts_text):.2f}")
print(f"  中位数: {np.median(unique_counts_text):.2f}")

In [ ]:
# 可视化每个维度的唯一值数量
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].bar(range(len(unique_counts_image)), unique_counts_image, alpha=0.7)
axes[0].axhline(y=len(df_image), color='r', linestyle='--', label=f'总样本数 ({len(df_image)})')
axes[0].set_title('图像嵌入 - 各维度唯一值数量', fontsize=12, fontweight='bold')
axes[0].set_xlabel('维度索引')
axes[0].set_ylabel('唯一值数量')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].bar(range(len(unique_counts_text)), unique_counts_text, alpha=0.7, color='orange')
axes[1].axhline(y=len(df_text), color='r', linestyle='--', label=f'总样本数 ({len(df_text)})')
axes[1].set_title('文本嵌入 - 各维度唯一值数量', fontsize=12, fontweight='bold')
axes[1].set_xlabel('维度索引')
axes[1].set_ylabel('唯一值数量')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. 检查是否存在数值碰撞

单个维度的唯一值少于样本数，可能的原因：

In [ ]:
# 检查某个维度的值分布
example_dim = 'emb_0'

print(f"以 {example_dim} 为例分析:\n")
print(f"图像嵌入:")
print(f"  总样本数: {len(df_image)}")
print(f"  {example_dim} 唯一值数量: {df_image[example_dim].nunique()}")
print(f"  差值: {len(df_image) - df_image[example_dim].nunique()}")

# 查看值的分布
value_counts = df_image[example_dim].value_counts()
duplicated_values = value_counts[value_counts > 1]

print(f"\n  有 {len(duplicated_values)} 个值出现了多次")
if len(duplicated_values) > 0:
    print(f"  出现次数最多的值:")
    for val, count in duplicated_values.head(5).items():
        print(f"    {val:.6f}: 出现 {count} 次")

## 5. 原因解释

### 🎯 核心解答：

**单个维度的唯一值少于样本数是正常的！**

原因如下：

1. **数值碰撞（Numerical Collision）**：
   - 浮点数精度有限（float32）
   - 不同电影在某个特定维度上可能恰好有相同或非常接近的值
   - 这是连续值空间中的正常现象

2. **嵌入的本质**：
   - 嵌入向量的**整体组合**才是唯一的
   - 单个维度只是整体的一个分量
   - 就像坐标系中，不同点可以有相同的 x 坐标，但 (x,y) 组合是唯一的

3. **多样性体现在整体**：
   - 50维空间中，即使每个维度只有100个不同值
   - 理论上可以表示 100^50 种不同的组合
   - 远超过3882个电影的需求

In [ ]:
# 验证：计算理论组合空间
print("=" * 80)
print("理论组合空间分析")
print("=" * 80)

avg_unique_per_dim_image = np.mean(unique_counts_image)
avg_unique_per_dim_text = np.mean(unique_counts_text)

print(f"\n图像嵌入:")
print(f"  平均每维唯一值: {avg_unique_per_dim_image:.0f}")
print(f"  维度数: {len(image_emb_cols)}")
print(f"  理论组合数: {avg_unique_per_dim_image:.0f}^{len(image_emb_cols)} (天文数字)")
print(f"  实际需要表示: {len(df_image)} 个电影")
print(f"  ✅ 空间充足度: 极其充足")

print(f"\n文本嵌入:")
print(f"  平均每维唯一值: {avg_unique_per_dim_text:.0f}")
print(f"  维度数: {len(text_emb_cols)}")
print(f"  理论组合数: {avg_unique_per_dim_text:.0f}^{len(text_emb_cols)} (天文数字)")
print(f"  实际需要表示: {len(df_text)} 个电影")
print(f"  ✅ 空间充足度: 极其充足")

## 6. 验证嵌入向量的实际多样性

In [ ]:
# 计算嵌入向量之间的相似度
from sklearn.metrics.pairwise import cosine_similarity

def analyze_embedding_diversity(embeddings, name, sample_size=500):
    """分析嵌入向量的多样性"""
    print("=" * 80)
    print(f"{name} - 多样性分析")
    print("=" * 80)
    
    # 随机采样以加快计算
    if len(embeddings) > sample_size:
        indices = np.random.choice(len(embeddings), sample_size, replace=False)
        sample_embeddings = embeddings[indices]
        print(f"\n使用 {sample_size} 个样本进行分析")
    else:
        sample_embeddings = embeddings
        print(f"\n使用全部 {len(embeddings)} 个样本")
    
    # 计算余弦相似度矩阵
    similarity_matrix = cosine_similarity(sample_embeddings)
    
    # 获取上三角（排除对角线）
    mask = np.triu(np.ones_like(similarity_matrix, dtype=bool), k=1)
    similarities = similarity_matrix[mask]
    
    print(f"\n相似度统计:")
    print(f"  均值: {similarities.mean():.4f}")
    print(f"  标准差: {similarities.std():.4f}")
    print(f"  最小值: {similarities.min():.4f}")
    print(f"  最大值: {similarities.max():.4f}")
    print(f"  中位数: {np.median(similarities):.4f}")
    
    # 绘制相似度分布
    plt.figure(figsize=(10, 6))
    plt.hist(similarities, bins=50, edgecolor='black', alpha=0.7)
    plt.axvline(similarities.mean(), color='r', linestyle='--', linewidth=2, label=f'均值: {similarities.mean():.4f}')
    plt.title(f'{name} - 嵌入向量余弦相似度分布', fontsize=14, fontweight='bold')
    plt.xlabel('余弦相似度')
    plt.ylabel('频数')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    if similarities.mean() < 0.5:
        print("\n✅ 嵌入向量具有良好的多样性（低相似度）")
    elif similarities.mean() < 0.8:
        print("\n⚠️  嵌入向量有一定相似性，但仍有区分度")
    else:
        print("\n❌ 嵌入向量相似度较高，多样性不足")

# 分析图像嵌入
analyze_embedding_diversity(image_embeddings, "图像嵌入")

In [ ]:
# 分析文本嵌入
analyze_embedding_diversity(text_embeddings, "文本嵌入")

## 7. 总结

### 📌 关键结论：

1. **单个维度的唯一值少是正常的**
   - 这不代表嵌入缺乏多样性
   - 多样性体现在50维的组合空间中

2. **整体嵌入向量是唯一的**
   - 每个电影都有独特的50维向量
   - 即使单个维度有重复值

3. **类比理解**：
   - 就像地球上的人，可能有相同的身高（单个维度）
   - 但身高+体重+年龄+...的组合是唯一的

4. **数学角度**：
   - 3873^50 >> 3882
   - 表示空间远大于实际需求

In [ ]:
print("\n" + "=" * 80)
print("最终验证")
print("=" * 80)

print(f"\n图像嵌入:")
print(f"  ✓ Movie ID 数量: {df_image['movie_id'].nunique()}")
print(f"  ✓ 唯一嵌入向量数量: {len(df_image[image_emb_cols].drop_duplicates())}")
print(f"  ✓ 是否一一对应: {df_image['movie_id'].nunique() == len(df_image[image_emb_cols].drop_duplicates())}")

print(f"\n文本嵌入:")
print(f"  ✓ Movie ID 数量: {df_text['movie_id'].nunique()}")
print(f"  ✓ 唯一嵌入向量数量: {len(df_text[text_emb_cols].drop_duplicates())}")
print(f"  ✓ 是否一一对应: {df_text['movie_id'].nunique() == len(df_text[text_emb_cols].drop_duplicates())}")

print("\n🎉 分析完成！嵌入数据结构正常，多样性充足！")